In [2]:
import subprocess
from pydub import AudioSegment
import math


# subprocess는 파이썬에서 command를 실행할 수있게 해준다.
# 동영상에서 오디오만 추출하는 명령어
# -vn: 영상은 무시하라는 옵션
# ffmpeg -i ./.cache/test2.mp4 -vn ./.cache/audio.mp3
def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


def cut_audio_in_chunks(audio_path, chunk_size, chunk_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_length = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_length)
    for i in range(chunks):
        start_time = i * chunk_length
        end_time = (i + 1) * chunk_length

        chunk = track[start_time:end_time]
        chunk.export(f"{chunk_folder}/chunk_{i}.mp3", format="mp3")

In [6]:
from openai import OpenAI

client = OpenAI()
transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=open("./.cache/chunks/chunk_1.mp3", "rb"),
)

Transcription(text="That's all about this React vs Flutter comparison. If you want to boost your skills with these frameworks, feel free to check out these links in the description. They might help you a lot. And if you like my content, please give it a thumb up. If you want to see more videos like this, make sure to hit that magic subscribe button. Thanks for watching and till next time. Transcribed by https://otter.ai")

In [9]:
# glob: 패턴을 사용하여 디렉토리 내부의 파일을 검색할 수 있도록 해준다.


import glob


def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcript = ""
    for file in files:
        with open(file, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
            )
            final_transcript += transcript.text
    with open(destination, "w") as file:
        file.write(final_transcript)


transcribe_chunks("./.cache/chunks", "./.cache/transcript.txt")